<a href="https://colab.research.google.com/github/MatteoGhizzi/MLPNS2021/blob/main/Lezione6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import scipy as sp
from scipy import stats

In [ ]:
sp.stats.norm.rvs(size=100)


array([ 0.28820567,  1.35083081, -0.01059785, -1.09785445,  0.92209653,
        0.01539784,  0.0628103 ,  2.13551658,  1.21729501,  1.21111035,
       -1.30335769,  0.27330507,  0.01046461, -0.63820433,  0.23721351,
        0.65647688,  0.41753362,  0.7150415 , -0.66519312, -0.2446004 ,
        0.60553464, -0.09385349, -1.50901813,  1.02969614,  0.45264675,
        0.74277272,  0.45470741, -0.25939598, -1.4439482 ,  1.10064688,
        0.70715972,  0.73580052,  2.39492702,  1.2299201 ,  0.51765014,
       -0.37711512, -0.46671693, -0.33292491, -0.73843143,  0.84892775,
       -0.79838011, -0.26128446, -0.00540678, -0.33008557, -1.7197987 ,
        0.61458387, -0.56665246,  0.17629905,  0.54929034, -0.46792158,
       -0.61861471,  0.27633581, -2.40343905,  0.30703047, -1.08196851,
       -1.2883005 ,  2.12338811, -0.64741857,  0.35787972,  0.83364301,
       -0.39093483, -0.10210796, -1.12297925,  1.33385124,  0.37058983,
       -0.60038703, -0.09839656,  2.48281406, -0.28103483, -0.49

In [ ]:
sp.stats.kstest?


In [ ]:
X = sp.stats.norm.rvs(size=100)

In [ ]:
sp.stats.kstest(X,'norm'). #statistic is the maximum distance between the 2 comulative distribution
# p value. ust be << than my thresh
# ripsonde alla domanda : bengono dalla stessa distribuzione ??

KstestResult(statistic=0.07900596445453356, pvalue=0.5450349912945878)

In [ ]:
Y = sp.stats.norm.rvs(size=1000)
sp.stats.kstest(Y,'norm')

KstestResult(statistic=0.023693043170701755, pvalue=0.6284469200886302)

In [ ]:
W = sp.stats.norm.rvs(size=10000)
sp.stats.kstest(W,'norm')

KstestResult(statistic=0.016301066979581003, pvalue=0.009839517221534453)

In [ ]:
import numpy as np


In [ ]:
K = np.random.binomial(10000,0.1,size=1000) # non funziona perchè non è standardizzata
sp.stats.kstest(K,'norm')

KstestResult(statistic=1.0, pvalue=0.0)

In [ ]:
#all the imports I needed
from __future__ import print_function, division
import pandas as pd
import pylab as pl
import scipy as sp
from scipy import stats
%pylab inline
pl.style.use("https://raw.githubusercontent.com/fedhere/DSPS/master/fbb.mplstyle")

Populating the interactive namespace from numpy and matplotlib


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/fedhere/MLPNS2021/main/NHRT/earthquakes.csv',sep=' ')

In [ ]:
data.describe


<bound method NDFrame.describe of        #YYY/MM/DD  HH:mm:SS.ss  ET  GT  ... Unnamed: 30  Unnamed: 31 NPH  NGRM
0      1984/01/01  01:30:57.52  eq NaN  ...         NaN          NaN NaN   NaN
1      1984/01/01  05:31:34.16  eq NaN  ...         NaN          NaN NaN   NaN
2      1984/01/01  06:59:05.14  eq NaN  ...         NaN          NaN NaN   NaN
3      1984/01/01  11:10:25.29  eq NaN  ...         NaN          NaN NaN   NaN
4      1984/01/01  19:38:19.79  eq NaN  ...         NaN          NaN NaN   NaN
...           ...          ...  ..  ..  ...         ...          ...  ..   ...
70793  2002/12/29  19:37:24.56  eq NaN  ...         NaN          NaN NaN   NaN
70794  2002/12/29  20:17:33.40  eq NaN  ...         NaN          NaN NaN   NaN
70795  2002/12/29  20:21:28.73  eq NaN  ...         NaN          NaN NaN   NaN
70796  2002/12/30  21:51:52.55  eq NaN  ...         NaN          NaN NaN   NaN
70797  2002/12/30  23:38:49.79  eq NaN  ...         NaN          NaN NaN   NaN

[70798 rows x 34 

In [ ]:
data.rename({'#YYY/MM/DD':'date','HH:mm:SS.ss':'time','Unnamed: 5':'mag'},axis=1)[['date','time','mag']]



,date,time,mag
0,1984/01/01,01:30:57.52,2.02
1,1984/01/01,05:31:34.16,2.12
2,1984/01/01,06:59:05.14,2.43
3,1984/01/01,11:10:25.29,2.07
4,1984/01/01,19:38:19.79,2.13
...,...,...,...
70793,2002/12/29,19:37:24.56,2.21
70794,2002/12/29,20:17:33.40,2.02
70795,2002/12/29,20:21:28.73,2.07
70796,2002/12/30,21:51:52.55,2.31


In [ ]:
for i in range(len(data['time'])):
  if data.iloc[i].time.endwith('60.00'):
   print(data.iloc[i].time)
   print(data.iloc[i].time[:6] + '59.99')
   data['time'][i] = data.iloc[i].time[:6] + '59.99'


KeyError: ignored